In [27]:
import time, os
import numpy as np
import pandas as pd
import igraph as ig
from tqdm.notebook import trange
from scipy.optimize import linear_sum_assignment as lap_solver
from fastdist import fastdist
from network_tracking import local_graph_comparison_score, list_to_str

work_dir = 'D:/Python Scripts/Mito/MitoTNT/test_data/'
data_dir = work_dir+'mitograph/'
input_dir = work_dir+'tracking_input/'
if not os.path.isdir(input_dir):
    os.mkdir(input_dir)

start_frame = 0
end_frame = len([frame for frame in os.listdir(data_dir) if 'FRAME' in frame])

track_dir = work_dir+'tracking_ouput/'
if not os.path.isdir(track_dir):
    os.mkdir(track_dir)

tracking_interval = 1
graph_matching_depth = 2
dist_exponent, top_exponent = 1, 1

min_track_size = 4
max_gap_size = 2
memory_efficient_gap_closing = True

In [2]:
## reload all tracks
all_tracks = np.load(track_dir+'/all_tracks.npy', allow_pickle=True)

# filter out very short tracks
very_short_tracks = []
for i in range(len(all_tracks)):
    if len(all_tracks[i][0]) < min_track_size:
        very_short_tracks.append(i)

all_tracks = np.delete(all_tracks, very_short_tracks, axis=0)
all_tracks = sorted(all_tracks, key=lambda track : track[0][0]) # sort by start frame
num_tracks = len(all_tracks)

print('Gap closing in progress ...')

# get track disps
all_track_disps = []
for t in all_tracks:
    track_coords = t[4] # use index for the coordinates
    all_track_disps.append([np.linalg.norm(track_coords[t+1]-track_coords[t]) for t in range(len(track_coords)-1)])

# store intermediate values
print('Data loading ...')
inputs = np.load(input_dir+'tracking_inputs.npz', allow_pickle=True)
full_graph_all_frames = inputs['full_graphs']
classic_graph_per_node_all_frames = inputs['classic_graphs_per_node']
segment_node_all_frames = inputs['segment_nodes']

all_track_assignments = {}
partition_start = 0

Gap closing in progress ...
Data loading ...


In [3]:
while partition_start < num_tracks:
    num_frame = end_frame-start_frame

    if not memory_efficient_gap_closing:
        partition_size = num_tracks
        overlap_size = 0
    else:
        partition_size = int(num_tracks / num_frame) * 30
        overlap_size =  int(num_tracks / num_frame) * 5

    partition_end = partition_start + partition_size
    if partition_end > num_tracks:
        partition_end = num_tracks
    print('Partition index:', partition_start, partition_end)

    track_cost_m_n = np.empty([partition_size, partition_size])
    track_cost_m_n[:] = np.nan

    for i in trange(partition_start, partition_end):

        track_frames_m, track_nodes_m, track_coords_m = all_tracks[i][0], all_tracks[i][1], all_tracks[i][4]
        end_frame_m, end_node_m, end_coord_m = track_frames_m[-1], track_nodes_m[-1], track_coords_m[-1]

        # load data

        classic_graphs_m = classic_graph_per_node_all_frames[end_frame_m]
        disps_m = all_track_disps[i]

        for j in range(i+1, partition_end): # no need to check index less than i because the start frame is sorted

            track_frames_n, track_nodes_n, track_coords_n = all_tracks[j][0], all_tracks[j][1], all_tracks[j][4]
            start_frame_n, start_node_n, start_coord_n = track_frames_n[0], track_nodes_n[0], track_coords_n[0]

            gap_size = start_frame_n - end_frame_m - 1

            # check only those within max gap size
            if 1 <= gap_size <= max_gap_size: # if gap == 1 it should have been linked before - so skip it

                # load data
                classic_graphs_n = classic_graph_per_node_all_frames[start_frame_n]

                # compute distance cutoff based on the two tracks
                disps_n = all_track_disps[j]
                comb_disps = disps_m + disps_n

                dist_cutoff = (gap_size + 1) * (3 * np.std(comb_disps)) ** 2

                # compute node-to-node distance
                dist = end_coord_m - start_coord_n
                dist_cost = np.sum(dist**2)

                # filter by distance cutoff
                if dist_cost > dist_cutoff:
                    continue

                # compute topology cost
                topology_cost = local_graph_comparison_score(graph_matching_depth, end_node_m, start_node_n, classic_graphs_m, classic_graphs_n)

                # assign g.c. cost
                track_cost_m_n[i-partition_start,j-partition_start] = dist_cost**dist_exponent * topology_cost**top_exponent

    # construct termination cost matrix
    track_cost_m_m = np.empty([partition_size, partition_size])
    track_cost_m_m[:] = np.nan

    for i in range(partition_size):
        row = track_cost_m_n[i,:]

        if np.isnan(row).all():
            track_cost_m_m[i,i] = 0 # must be assigned to itself since all other nodes exceed max radius
        else:
            min_cost = np.nanmin(row)
            track_cost_m_m[i,i] = 1.5 * min_cost
            # track_cost_m_m[i,i] = np.nanpercentile(row, 90, interpolation='midpoint')

    # assemble into one matrix
    track_cost_matrix = np.concatenate((track_cost_m_n, track_cost_m_m), axis=1)
    track_cost_matrix[np.isnan(track_cost_matrix)] = np.inf

    # evaluate memory usage
    print('\n%d MB' % (track_cost_matrix.nbytes / 1024**2), '\n')

    # solve LAP and store linking results
    assignment = lap_solver(track_cost_matrix)[1]

    linked, terminated, initiated = [], [], []
    for i in range(len(assignment)):
        if assignment[i] < partition_size:
            linked.append([i, assignment[i]]) # first being index for frame t and second for frame t+tracking_interval
        else:
            terminated.append(i)

    for pair in linked:
        all_track_assignments[partition_start + pair[0]] = partition_start + pair[1] # offset by start index of the partition

    # go to next partition
    partition_start = partition_start + partition_size - overlap_size

Partition index: 0 4260


  0%|          | 0/4260 [00:00<?, ?it/s]


276 MB 

Partition index: 3550 7810


  0%|          | 0/4260 [00:00<?, ?it/s]


276 MB 

Partition index: 7100 11360


  0%|          | 0/4260 [00:00<?, ?it/s]


276 MB 

Partition index: 10650 13225


  0%|          | 0/2575 [00:00<?, ?it/s]


276 MB 



In [4]:
# convert dictionary to array and overwrite assignment by next partition's assignment for the overlapped region
linked_tracks = np.zeros([len(all_track_assignments.keys()), 2], dtype=int)
for i, a in enumerate(all_track_assignments.keys()):
    linked_tracks[i,0] = a
    linked_tracks[i,1] = all_track_assignments[a]

In [7]:
linked_tracks

array([[    0,  1458],
       [    2,  1529],
       [    3,  1483],
       ...,
       [12416, 13187],
       [12419, 13223],
       [12420, 13199]])

In [8]:
# combine tracks for gap closing
print('Start combining closed tracks ...')
if linked_tracks.shape[0] > 0:
    linked_tracks_for_update = linked_tracks.copy() # used to record appended tracks
    tracks_of_track = [] # list of linked tracks
    all_linked_tracks = [] # record tracks that are closed

    # recursive function for finding linked tracks
    def find_all_linked_tracks(tracks, track_id):
        for i in range(0, len(linked_tracks_for_update)):
            if linked_tracks_for_update[i,0] == track_id: # find the first track
                tracks.append(linked_tracks_for_update[i,1])
                linked_tracks_for_update[i,0] = -1 # note that the track is already appended
                find_all_linked_tracks(tracks, linked_tracks_for_update[i,1]) # go to the next track and find linked_tracks track

    # for each track find the series of linked tracks
    for index in trange(len(linked_tracks)):
        track_id = linked_tracks[index,0]
        if track_id in linked_tracks_for_update[:,0]:
            tracks = [track_id]
            find_all_linked_tracks(tracks, track_id)
            tracks_of_track.append(tracks)
            all_linked_tracks += tracks

    # concatenate data for closed tracks
    all_closed_tracks = []
    for tot in tracks_of_track:
        all_closed_tracks.append([sum([all_tracks[t][0] for t in tot], []),
                                  sum([all_tracks[t][1] for t in tot], []),
                                  sum([all_tracks[t][2] for t in tot], []),
                                  sum([all_tracks[t][3] for t in tot], []),
                                  sum([all_tracks[t][4] for t in tot], []),
                                  sum([all_tracks[t][5] for t in tot], []),
                                  sum([all_tracks[t][6] for t in tot], [])])

    # add unclosed tracks back
    for t in range(num_tracks):
        if t not in all_linked_tracks:
            all_closed_tracks.append(all_tracks[t].tolist())

    # sort tracks
    sort_by_length = sorted(all_closed_tracks, key=lambda track : len(track[0]), reverse=True) # first sort by size of track
    sort_by_start = sorted(sort_by_length, key=lambda track : track[0][0]) # then sort by start frame
    all_closed_tracks = sort_by_start

else:
    all_closed_tracks = all_tracks

Start combining closed tracks ...


  0%|          | 0/4696 [00:00<?, ?it/s]

In [12]:
tot = tracks_of_track[0]
[sum([all_tracks[t][0] for t in tot], []),
                                  sum([all_tracks[t][1] for t in tot], []),
                                  sum([all_tracks[t][2] for t in tot], []),
                                  sum([all_tracks[t][3] for t in tot], []),
                                  sum([all_tracks[t][4] for t in tot], []),
                                  sum([all_tracks[t][5] for t in tot], []),
                                  sum([all_tracks[t][6] for t in tot], [])]

[[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18],
 [3, 3, 4, 1, 7, 10, 15, 11, 15, 13, 6, 5, 4, 1328, 7, 5, 7],
 [243,
  268,
  237,
  271,
  259,
  266,
  260,
  258,
  274,
  268,
  244,
  274,
  274,
  262,
  259,
  259,
  255],
 [3, 2, 3, 1, 5, 5, 5, 3, 1, 0, 5, 3, 2, 2, 1, 4, 0],
 [array([158., 125.,  42.]),
  array([155., 124.,  42.]),
  array([155., 122.,  42.]),
  array([157., 121.,  43.]),
  array([160., 122.,  42.]),
  array([158., 127.,  41.]),
  array([158., 128.,  41.]),
  array([157., 129.,  41.]),
  array([156., 129.,  41.]),
  array([152., 129.,  40.]),
  array([151., 129.,  42.]),
  array([150., 125.,  42.]),
  array([150., 123.,  42.]),
  array([151., 124.,  42.]),
  array([148., 126.,  41.]),
  array([146., 127.,  42.]),
  array([147., 127.,  41.])],
 [851.5,
  1017.33333,
  1406.66667,
  923.0,
  1921.33333,
  2066.66667,
  1721.5,
  1152.5,
  1813.0,
  1071.0,
  1651.33333,
  1592.66667,
  1305.5,
  1499.66667,
  1222.16667,
  2050.0,
  1371.33333],
 [1

In [18]:
np.save(track_dir+'all_closed_tracks.npy', np.array(all_closed_tracks, dtype=object))

print('\nNumber of tracks and average track length before gap closing:',
      len(all_tracks), np.mean([len(track[0]) for track in all_tracks]))

print('\nNumber of tracks and average track length after gap closing:',
      len(all_closed_tracks), np.mean([len(track[0]) for track in all_closed_tracks]))


Number of tracks and average track length before gap closing: 13225 8.701701323251418

Number of tracks and average track length after gap closing: 8529 13.492789307069996


In [ ]:
### Save tracks in the form of one node per row ###
tracks = pd.DataFrame(columns={'frame_id', 'unique_node_id', 'frame_node_id', 'frame_seg_id', 'frame_frag_id', 'x', 'y', 'z','intensity','width'})
tracks = tracks[['frame_id', 'unique_node_id', 'frame_node_id', 'frame_seg_id', 'frame_frag_id', 'x', 'y', 'z','intensity','width']] # reorder the columns

df_index = 0
for track_id, track in enumerate(all_closed_tracks):
    track_frames, track_nodes, track_segs, track_frags, track_coords, track_ints, track_widths = track[0], track[1], track[2], track[3], track[4], track[5], track[6]

    for f in range(len(track_frames)):
        x, y, z = track_coords[f][0], track_coords[f][1], track_coords[f][2]
        tracks.loc[df_index] = {'frame_id': track_frames[f], 'unique_node_id': track_id,
                        'frame_node_id': track_nodes[f], 'frame_seg_id': track_segs[f], 'frame_frag_id': track_frags[f],
                        'x': x, 'y': y, 'z': z, 'intensity': track_ints[f], 'width': track_widths[f]}
        df_index += 1

tracks.sort_values(['unique_node_id'], inplace=True, ignore_index=True)
tracks.to_csv(track_dir+'tracks.csv', index=False)
### Done saving ###

In [ ]:
### Save tracks and connected nodes using unique indexing ###
new_tracks = pd.DataFrame()
for frame in trange(start_frame, end_frame, tracking_interval):

    full_graph = full_graph_all_frames[frame]
    tracks_frame = tracks[tracks['frame_id'] == frame]

    tracked_frame_nodes = tracks_frame['frame_node_id'].astype('int').tolist()
    unique_nodes = tracks_frame['unique_node_id'].astype('int').tolist()
    frame_to_unique = {tracked_frame_nodes[i]:unique_nodes[i] for i in range(len(tracks_frame))}

    def find_connected_unique_nodes(this_node, visited_nodes):

        neighs = full_graph.neighbors(this_node)
        for visited_node in visited_nodes:
            if visited_node in neighs:
                neighs.remove(visited_node)

        if len(neighs) == 0:
            # print ('no neighbor for node',visited_nodes[0])
            return

        visited_nodes.append(this_node)

        for neigh in neighs:
            # if the frame node is tracked for this frame, add to list
            if neigh in tracked_frame_nodes:
                connected_unique_nodes.append(frame_to_unique[neigh])
                # print('add',neigh,'for node',visited_nodes[0])
            else:
                find_connected_unique_nodes(neigh, visited_nodes)
        return

    all_connected_unique_nodes = []
    for node in tracked_frame_nodes:
        connected_unique_nodes = []
        find_connected_unique_nodes(node, [node])
        all_connected_unique_nodes.append(connected_unique_nodes)

    tracks_frame.insert(2, 'connected_unique_node_id', [list_to_str(a) for a in all_connected_unique_nodes])
    new_tracks = pd.concat([new_tracks, tracks_frame]) # accumulate tracks from each frame

# reorder the columns
new_tracks = new_tracks[['frame_id', 'unique_node_id', 'frame_node_id', 'frame_seg_id', 'frame_frag_id', 'connected_unique_node_id', 'x', 'y', 'z', 'intensity', 'width']]
new_tracks.to_csv(track_dir+'tracks_with_connectivity_unique_index.csv', index=False)

### Done saving ###


### Save tracks and connected nodes using frame indexing ###
new_tracks = pd.DataFrame()
for frame in trange(start_frame, end_frame, tracking_interval):

    full_graph = full_graph_all_frames[frame]
    cc = full_graph.components()

    segment_nodes = segment_node_all_frames[frame]
    branching_nodes = []
    for i in range(len(full_graph.vs)):
        if full_graph.vs[i].degree() > 2:
            branching_nodes.append(i)
    node_to_segment = {}
    for segment_id, segment in enumerate(segment_nodes): # segment consists of of segment nodes
        for b in segment:
            if b in branching_nodes:
                node_to_segment[b] = np.nan
            else:
                node_to_segment[b] = segment_id

    coords = full_graph.vs['coordinate']
    intensities, widths = full_graph.vs['intensity'], full_graph.vs['width']
    tracks_frame = tracks[tracks['frame_id'] == frame]

    # add new rows for untracked nodes
    untracked = []
    tracked_frame_node_id = tracks_frame['frame_node_id'].astype('int').tolist()
    for node in range(len(coords)):
        if node not in tracked_frame_node_id:
            coord = coords[node]
            x, y, z = coord[0], coord[1], coord[2]
            untracked.append({'frame_id': frame, 'unique_node_id': 'untracked',
                              'frame_node_id': node, 'frame_seg_id': node_to_segment[node], 'frame_frag_id': cc.membership[node],
                              'x': x, 'y': y, 'z': z,
                              'intensity':intensities[node], 'width':widths[node]})

    untracked_tracks = pd.DataFrame.from_dict(untracked)
    tracks_frame = pd.concat([tracks_frame, untracked_tracks])

    # add new column with neighbor nodes for each node
    all_connected_frame_nodes = []
    tracked_frame_node_id = tracks_frame['frame_node_id'].astype('int').tolist()
    for node in tracked_frame_node_id:
        neighs = full_graph.neighbors(int(node))
        all_connected_frame_nodes.append(neighs)

    tracks_frame.insert(2, 'connected_frame_node_id', [list_to_str(a) for a in all_connected_frame_nodes])

    # accumulate tracks from each frame
    new_tracks = pd.concat([new_tracks, tracks_frame])

# reorder the columns
new_tracks = new_tracks[['frame_id', 'unique_node_id', 'frame_node_id', 'frame_seg_id', 'frame_frag_id', 'connected_frame_node_id', 'x', 'y', 'z', 'intensity', 'width']]
new_tracks.to_csv(track_dir+'tracks_with_connectivity_frame_index.csv', index=False)
### Done saving ###